**Volatility Surface**




The Black-Scholes formula is a mathematical model used to estimate the fair price of a call or put option. There are five inputs: current stock price, strike price, time to expiration, risk-free interest rate, and volatility.

### Black-Scholes Formula

$C = S_0 N(d_1) - K e^{-rT} N(d_2)$

Where:

$d_1 = \frac{\ln(S_0/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$

$d_2 = d_1 - \sigma\sqrt{T}$

And:

*   $C$: Call option price
*   $S_0$: Current stock price (spot price)
*   $K$: Option strike price
*   $T$: Time to expiration (in years)
*   $r$: Risk-free interest rate (annualized)
*   $\sigma$: Volatility of the asset (annualized standard deviation of returns)
*   $N()$: Cumulative standard normal distribution function
*   $e$: Euler's number (the base of the natural logarithm)


In [1]:
from scipy.stats import norm
import numpy as np

def black_scholes_call(S0: float, K: float, T:float, r: float, sigma: float, q: float = 0.0) -> float:

  # check the expiration date
  if T<= 0.0:
    return max(S0 - K, 0.0)

  # make sure sigma is positive and more than zero
  sigma = max(float(sigma), 1e-12)

  # calculate d1 and d2
  sigma_sqrt_T = sigma * np.sqrt(T)
  d1 = (np.log(S0/K)+ (r - q + 0.5 * sigma * sigma) * T) / sigma_sqrt_T
  d2 = d1 - sigma_sqrt_T

  discounted_spot = S0 * np.exp(-q * T)
  discounted_strike = K * np.exp(-r * T)

  # calculate the call option price
  option_price = float(discounted_spot * norm.cdf(d1) - discounted_strike * norm.cdf(d2))

  return option_price


### Mimic the stock market volatility

Here we create a true volatility surface.
- skew/smile across volatility surface (downside tends to have higher implied volatility)
- term structure across maturities

In [30]:
def sigma_true(S0: float, K: float, T: float) -> float:

    m = K / S0
    base = 0.20

    # smile curvature: stronger for short maturity
    smile_strength = 0.16 / np.sqrt(np.maximum(T, 1e-6))
    smile = smile_strength * (m - 1.0) ** 2

    # skew: tilt so low strikes have higher vol
    skew = -0.10 * (m - 1.0)

    # term structure: extra short-dated vol that decays with T
    term = 0.03 * np.exp(-1.5 * T)

    sig = base + smile + skew + term
    return float(np.clip(sig, 0.05, 0.90))

def generate_synthetic_market_prices(
    S0: float,
    r: float,
    q: float,
    strikes: np.ndarray,
    maturities: np.ndarray,
    noise_std: float = 0.0,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    market_prices = pd.DataFrame(index=strikes, columns=maturities, dtype=float)
    true_vol_grid = pd.DataFrame(index=strikes, columns=maturities, dtype=float)

    for K in strikes:
        for T in maturities:
            sig = sigma_true(S0, float(K), float(T))
            C = black_scholes_call(S0, float(K), float(T), r, sig, q=q)

            # optional multiplicative noise
            eps = np.random.normal(0.0, noise_std) if noise_std > 0 else 0.0
            C_mkt = C * (1.0 + eps)

            # enforce lower bound: intrinsic under forwards
            intrinsic = max(S0 * np.exp(-q * T) - float(K) * np.exp(-r * T), 0.0)
            C_mkt = max(float(C_mkt), float(intrinsic))

            market_prices.loc[K, T] = C_mkt
            true_vol_grid.loc[K, T] = sig

    return market_prices, true_vol_grid

In [32]:
# market inputs
S0 = 100.0
r = 0.04
q = 0.00

# Strikes and maturities
strikes = np.array([70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 130], dtype=float)
strikes = np.sort(strikes)

maturities = np.array([0.10, 0.25, 0.50, 0.75, 1.00, 1.50, 2.00], dtype=float)
maturities = np.sort(maturities)

NOISE_STD = 0.00  # noise control

market_prices,true_vol_grid= generate_synthetic_market_prices(
    S0=S0, r=r, q=q, strikes=strikes, maturities=maturities, noise_std=NOISE_STD
)

print("Market call prices: ")
display(market_prices.round(6))

Market call prices: 


,0.10,0.25,0.50,0.75,1.00,1.50,2.00
70.0,30.279585,30.712221,31.512460,32.369573,33.243722,34.983994,36.680252
75.0,25.300518,25.795849,26.753227,27.760606,28.765224,30.719388,32.587338
80.0,20.327323,20.941264,22.124423,23.309919,24.454882,26.622640,28.653138
85.0,15.388367,16.235150,17.714543,19.091366,20.373792,22.737260,24.910284
90.0,10.603499,11.837371,13.641316,15.192496,16.590352,19.109439,21.392691
95.0,6.309808,7.978219,10.036983,11.704113,13.173067,15.784296,18.133379
100.0,3.046651,4.892054,7.018128,8.703964,10.180473,12.801068,15.161735
105.0,1.138335,2.701948,4.649481,6.238841,7.650386,10.187991,12.500581
110.0,0.327620,1.344808,2.920230,4.312000,5.591801,7.958001,10.163498
115.0,0.076418,0.611402,1.747730,2.881806,3.982493,6.106407,8.152973


### Implied volatility via a root-finder (bisection)

implied volatility solves => f(σ) = BS_price(σ) - market_price

Why bisesction is a good strategy;
- it's simple, stable, and hard to break

In [33]:
def implied_vol_bisection(
    C_mkt: float,
    S0: float,
    K: float,
    T: float,
    r: float,
    q: float = 0.0,
    sigma_low: float = 1e-8, # tiny value
    sigma_high: float = 2.0, # max value
    tol: float = 1e-12,
    max_iter: int = 250, # max number of guesses
) -> float:

    if T <= 0.0:
        return np.nan

    C_mkt = float(C_mkt)

    def f(sig: float) -> float:
        return black_scholes_call(S0, K, T, r, sig, q=q) - C_mkt

    lo = float(sigma_low)
    hi = float(sigma_high)
    f_lo = f(lo) # BS(0%) - $7.84 = negative
    f_hi = f(hi) # BS(200%) - $7.84 = positive

    # expand hi until we bracket a sign change or give up
    tries = 0
    while f_lo * f_hi > 0.0 and tries < 25:
        hi *= 1.5
        f_hi = f(hi)
        tries += 1

    if f_lo * f_hi > 0.0:
        return np.nan

    for _ in range(max_iter): # bisection
        mid = 0.5 * (lo + hi)
        f_mid = f(mid)

        if abs(f_mid) < tol or (hi - lo) < tol:
            return float(mid)

        if f_lo * f_mid <= 0.0:
            hi = mid
            f_hi = f_mid
        else:
            lo = mid
            f_lo = f_mid

    return float(mid)

### Implied volatility grid

Create a full table of implied vols (strike by maturity), which is used for 3D rendering

In [34]:
implied_surface = pd.DataFrame(index=strikes, columns=maturities, dtype=float)

for K in strikes:
    for T in maturities:
        implied_surface.loc[K, T] = implied_vol_bisection(
            C_mkt=float(market_prices.loc[K, T]),
            S0=S0, K=float(K), T=float(T),
            r=r, q=q
        )

print("Implied vol surface recovered from prices (%):")
display((implied_surface * 100).round(6))

Implied vol surface recovered from prices (%):


,0.10,0.25,0.50,0.75,1.00,1.50,2.00
70.0,30.135804,27.941868,26.453567,25.636726,25.10939,24.491953,24.167595
75.0,28.244402,26.561868,25.331313,24.628658,24.16939,23.632694,23.356468
80.0,26.605982,25.341868,24.322196,23.712966,23.30939,22.838755,22.601910
85.0,25.220544,24.281868,23.426217,22.889650,22.52939,22.110136,21.903920
90.0,24.088088,23.381868,22.643374,22.158709,21.82939,21.446837,21.262498
95.0,23.208615,22.641868,21.973668,21.520145,21.20939,20.848858,20.677645
100.0,22.582124,22.061868,21.417100,20.973957,20.66939,20.316198,20.149361
105.0,22.208615,21.641868,20.973668,20.520145,20.20939,19.848858,19.677645
110.0,22.088088,21.381868,20.643374,20.158709,19.82939,19.446837,19.262498
115.0,22.220544,21.281868,20.426217,19.889650,19.52939,19.110136,18.903920


### Smooth values (bicubic spline)




In [36]:
from scipy.interpolate import RectBivariateSpline

moneyness = (strikes / S0).astype(float)
Z_grid = implied_surface.to_numpy(dtype=float)

SPLINE_SMOOTHING = 0.0 if NOISE_STD == 0.0 else 1e-4

spline = RectBivariateSpline(moneyness, maturities, Z_grid, kx=3, ky=3, s=SPLINE_SMOOTHING)
print("Spline fitted. s =", SPLINE_SMOOTHING)

Spline fitted. s = 0.0


### 3D volatility surface
- x: strike
- y: time to maturity
- z: implied volatility in percent

In [37]:
# strike: descending
strike_grid = np.linspace(float(strikes.max()), float(strikes.min()), 200)

# time: ascending values
t_grid = np.linspace(float(maturities.min()), float(maturities.max()), 200)

K_grid, T_grid = np.meshgrid(strike_grid, t_grid)
M_grid = K_grid / S0

Z_vol = spline.ev(M_grid.ravel(), T_grid.ravel()).reshape(K_grid.shape)
Z_vol = np.clip(Z_vol, 0.01, 3.0)

surface = go.Surface(
    x=T_grid,               # Time (years)
    y=K_grid,               # Strike
    z=Z_vol * 100.0,        # Vol (%)
    opacity=0.92,
    colorbar=dict(title="Implied vol (percent)", len=0.75),
    name="Spline surface",
)

dots_x, dots_y, dots_z = [], [], []
for T in np.sort(maturities):                 # keep time ascending in points
    for K in strikes:
        dots_x.append(float(T))
        dots_y.append(float(K))
        dots_z.append(float(implied_surface.loc[K, T]) * 100.0)

dots = go.Scatter3d(
    x=dots_x, y=dots_y, z=dots_z,
    mode="markers",
    marker=dict(size=4),
    name="IV grid points",
)

fig = go.Figure(data=[surface, dots])
fig.update_layout(
    title=dict(
        text="Implied Volatility Surface (x=Time, y=Strike, z=Vol)",
        x=0.5, xanchor="center",
        pad=dict(t=2, b=2),
    ),
    height=820,
    width=1120,

    # remove whitespace around the figure
    margin=dict(l=0, r=0, t=35, b=0),

    scene=dict(
        # make the 3D scene use the full canvas
        domain=dict(x=[0.0, 1.0], y=[0.0, 1.0]),

        # time axis: show 0.5 at the "start" by reversing visual direction
        xaxis=dict(title="Time to maturity (years)", autorange="reversed"),

        yaxis=dict(title="Strike (K)"),
        zaxis=dict(title="Implied volatility (percent)"),
        camera=dict(eye=dict(x=1.35, y=1.35, z=0.9)),
    ),
)

fig.show()